前面讲解的都是Legacy Chains，下面看最新的基于LCEL构建的Chains。
* create_sql_query_chain
* create_stuff_documents_chain
* create_openai_fn_runnable
* create_structured_output_runnable
* load_query_constructor_runnable
* create_history_aware_retriever
* create_retrieval_chain


# 1、create_sql_query_chain的使用
`pip install pymysql`

举例1：

In [2]:
# pip install -U langchain langchain-community langchain-openai
from langchain_openai import ChatOpenAI
from langchain_classic.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase

# 测试连接本地的mysql数据库
db_user = "root"
db_password = "123456"
db_host = "localhost" #或 127.0.0.1
db_port = "3306"
db_database = "ai-agent"
# mysql+pymysql://用户名:密码@ip地址:端口号/数据库名
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_database}")

print("操作的是哪种数据库：",db.dialect)
print("获取数据库中的表：",db.get_usable_table_names())

#执行查询操作
res = db.run("SELECT COUNT(*) FROM employees")
print(res)

操作的是哪种数据库： mysql
获取数据库中的表： ['chat_message', 'employee']


ProgrammingError: (pymysql.err.ProgrammingError) (1146, "Table 'ai-agent.employees' doesn't exist")
[SQL: SELECT COUNT(*) FROM employees]
(Background on this error at: https://sqlalche.me/e/20/f405)

举例2：chain的使用

In [4]:
# pip install -U langchain langchain-community langchain-openai
from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase

#1、获取mysql数据库的连接
# 测试连接本地的mysql数据库
db_user = "root"
db_password = "123456"
db_host = "localhost" #或 127.0.0.1
db_port = "3306"
db_database = "ai-agent"
# mysql+pymysql://用户名:密码@ip地址:端口号/数据库名
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_database}")

# 2、获取大语言模型
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
chat_model = ChatOpenAI(model="gpt-4o-mini")

# 3、创建create_sql_query_chain的实例
chain = create_sql_query_chain(chat_model, db)
# response = chain.invoke({"question": "数据表employees中一共有多少个员工？",
#                          "table_names_to_use":["employees"]})
# print(response)

response = chain.invoke({"question": "数据表employee中薪资最高的员工信息",
                         "table_names_to_use":["employee"]})
print(response)

SQLQuery: SELECT `name`, `gender`, `birth_date`, `hire_date`, `department`, `position`, `salary` FROM `employee` ORDER BY `salary` DESC LIMIT 1;


# 2、create_stuff_documents_chain的使用

In [5]:
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document

# 定义提示词模板
prompt = PromptTemplate.from_template("""
基于文档{docs}中说的情况，香蕉是什么颜色的？
""")

# 创建链
llm = ChatOpenAI(model="gpt-4o-mini")
chain = create_stuff_documents_chain(llm, prompt, document_variable_name="docs")

# 文档输入
docs123 = [
    Document(
        page_content="苹果，学名Malus pumila Mill.，别称西洋苹果、柰，属于蔷薇科苹果属的植物。苹果是全球最广泛种植和销售的水果之一，具有悠久的栽培历史和广泛的分布范围。苹果的原始种群主要起源于中亚的天山山脉附近，尤其是现代哈萨克斯坦的阿拉木图地区，提供了所有现代苹果品种的基因库。苹果通过早期的贸易路线，如丝绸之路，从中亚向外扩散到全球各地。"
    ),
    Document(
        page_content="香蕉是白色的水果，主要产自热带地区。"

    ),
    Document(
        page_content="蓝莓是蓝色的浆果，含有抗氧化物质。"

    )
]
# 执行摘要
chain.invoke({"docs": docs123})

'根据您提供的信息，香蕉是白色的水果。通常情况下，香蕉在成熟时是黄色的，但在您所述的文档中指出它是白色的。请注意，这种颜色描述可能是特定于某种品种或在特定情况下的描述。'